# Installation & Maintenance





## Trust the PSGallery Repository

In [ ]:
Set-PSRepository -name PSGallery -InstallationPolicy Trusted

## Install or Update

In [ ]:
if (-not (Get-Module -ListAvailable -Name dbatools)) {
    Write-Output "dbatools is missing, let's install!"
    Install-Module -Name dbatools -Scope CurrentUser;
    Write-Output "dbatools is installed";
} else {
    Write-Output "You're already in the club, let's get you updated."
    Update-Module -Name dbatools;
    Write-Output "dbatools is up to date";
}

# Global Variables

In [ ]:
$Sql17 = "flexo\Sql17";
$Sql19 = "flexo\Sql19";

# Backup & Restore
## Basic Backup

In [ ]:
$BackupPath = "C:\SQL\Backup\";
$BackupParams = @{
    SqlInstance = $Sql17;
    Path = $BackupPath;
    Database = "Satellites";
    CreateFolder = $true;
}
$BackupResult = Backup-DbaDatabase @BackupParams;
$BackupResult | Format-List -Property *;
# Save this for later
$SingleBackupFile = $BackupResult.BackupPath;
Get-ChildItem $BackupPath | Format-Table -AutoSize;
Get-ChildItem "C:\SQL\Backup\Satellites" -Recurse | Sort-Object -Property LastWriteTime -Descending;

## More Advanced

- Copy-only
- Multiple files
- Compression
- Checksum
- Adjust MaxTransferSize and BufferCount
- Custom timestamp format

See <a href="https://sirsql.net/2012/12/13/20121212automated-backup-tuning/" data-href="https://sirsql.net/2012/12/13/20121212automated-backup-tuning/" title="https://sirsql.net/2012/12/13/20121212automated-backup-tuning/">https://sirsql.net/2012/12/13/20121212automated-backup-tuning/</a> for scripts to test your own backup performance

In [ ]:
$BackupParams = @{
    SqlInstance = $Sql17;
    Path = $BackupPath;
    Database = "Satellites";
    CreateFolder = $true;
    CopyOnly = $true;
    Type = "Full";
    FileCount =  7;
    CompressBackup = $true;
    Checksum = $true;
    BufferCount =1000;
    MaxTransfersize=65536;
    TimeStampFormat = "yyyy-MMM-dd HH.mm.ss";
}
$BackupResult = Backup-DbaDatabase @BackupParams;
Get-ChildItem -Path "C:\SQL\Backup\Satellites" -Recurse | Sort-Object -Property LastWriteTime -Descending;
$BackupResult | Format-List -Property *;

## Just Looking!

In [ ]:
$BackupParams = @{
    SqlInstance = $Sql17;
    Path = $BackupPath;
    Database = "Satellites";
    CreateFolder = $true;
    CopyOnly = $true;
    Type = "Full";
    FileCount =  7;
    CompressBackup = $true;
    Checksum = $true;
    BufferCount =1000;
    MaxTransfersize=65536;
    OutputScriptOnly = $true;
}
Backup-DbaDatabase @BackupParams;

## Restoring the Latest Backup

In [ ]:
$RestoreParams = @{
    SqlInstance = $Sql19;
    Path = "C:\SQL\Backup\Satellites";
    Database = "Satellites19";
}
$RestoreResult = Restore-DbaDatabase @RestoreParams;
$RestoreResult | Format-List -Property *;

## Restoring a Specific Backup

In [ ]:
$RestoreParams = @{
    SqlInstance = $Sql19;
    Path = $SingleBackupFile;
    Database = "Satellites19";
    WithReplace = $true;
}
$RestoreResult = Restore-DbaDatabase @RestoreParams;
$RestoreResult | Format-List -Property *;

## Just Looking (again)!

Maybe I'm just looking to learn how to construct a <code>RESTORE DATABASE</code> SQL statement. Other times, I just want to review the statement before it's executed.

In [ ]:
$RestoreParams = @{
    SqlInstance = $Sql19;
    Path = "c:\SQL\Backup\Satellites"
    Database = "Satellites19";
    WithReplace = $true;
    OutputScriptOnly = $true;
}
Restore-DbaDatabase @RestoreParams;

# Basic Backup Information
* Where do backups go?
* Do we have backups?
* Do they get compressed?

In [ ]:
$SqlInstance = "localhost\sql17";
Get-DbaDefaultPath -SqlInstance $SqlInstance;

### Open an Explorer window to the backup location

In [ ]:
$SqlInstance = "localhost\sql17";
Get-DbaDefaultPath -SqlInstance  $SqlInstance | Select-Object -ExpandProperty Backup | Invoke-Item;

### Check & Set Backup Compression

In [ ]:
$SqlInstance = "localhost\sql17";
Get-DbaSpConfigure -SqlInstance $SqlInstance -Name DefaultBackupCompression;

In [ ]:
$SqlInstance = "localhost\sql17";
Set-DbaSpConfigure -SqlInstance $SqlInstance -Name DefaultBackupCompression -Value 1;

## Let's Work With Backups!

When did we last back up our databases?

In [ ]:
Get-DbaLastBackup -SqlInstance $SqlInstance | Format-Table -AutoSize;

### Backup history for one database

In [ ]:
Get-DbaDbBackupHistory -SqlInstance $SqlInstance -Database DBA

### Recent Backups

In [ ]:
Get-DbaDbBackupHistory -SqlInstance $SqlInstance -IncludeCopyOnly -Since (Get-Date).AddDays(-7) -DeviceType Disk | Sort-Object -property Start;

Let's send that to Excel instead

In [ ]:
Get-DbaDbBackupHistory -SqlInstance $SqlInstance -IncludeCopyOnly -Since (Get-Date).AddDays(-7) -DeviceType Disk | Sort-Object -property Start | Export-Excel -Path C:\users\andy\documents\BackupHistory.xlsx -AutoSize -FreezeTopRow -Show;

# Ad hoc Backups

In [ ]:
$Sql17 = "localhost\sql17";
Get-DbaDefaultPath -SqlInstance $Sql17;
$BackupParameters = @{
    SqlInstance = $Sql17;
    Type = "Full";
    Database = "imdb";
    CopyOnly = $true;
    CompressBackup = $true;

}
Backup-DbaDatabase @BackupParameters;

### Automated Backups

In [ ]:
$SqlInstance = "localhost\sql17";
Install-DbaMaintenanceSolution -SqlInstance $SqlInstance -Solution All -Database DBAThings -CleanupTime 97 -InstallJobs -ReplaceExisting -LogToTable;

$EveryFiveMinutes = New-DbaAgentSchedule -SqlInstance $SqlInstance -Schedule "Five Minutes" -FrequencyType Daily -FrequencyInterval 1 -FrequencySubdayType Minutes -FrequencySubdayInterval 5 -Force;
$EveryFifteenMinutes = New-DbaAgentSchedule -SqlInstance $SqlInstance -Schedule "Five Minutes" -FrequencyType Daily -FrequencyInterval 1 -FrequencySubdayType Minutes -FrequencySubdayInterval 15 -Force;

Backup-DbaDatabase -SqlInstance localhost\sql17 -Type Full